In [2]:
from api import MovieApi 
import pandas as pd
import numpy as np

Gather data needed for movie analysis

In [3]:
# get movie data
session = MovieApi()
movies = session.get_top_rated_movies().json()

In [17]:
# read movie data from "db" 
top_movies = pd.read_csv('./top_rated_movies33.csv').drop(columns= ['Unnamed: 0'],axis=1)
top_movies.shape

(10000, 14)

clean data gathered to prep for analysis

In [20]:
# get movie genre info #
genres = session.get_movie_genre_list().json()['genres']
new_genre_columns = [str(g['id']) + '_' + str(g['name']) for g in genres] # checkout list comprehesion https://www.digitalocean.com/community/tutorials/understanding-list-comprehensions-in-python-3
# new_genre_columns = [str(g['id']) for g in genres] # just makes the columns name the genre_id

# remove spaces in genre names #
for i, val in enumerate(new_genre_columns):
    new_genre_columns[i] = val.replace(' ','_')
# new_genre_columns

# create new genre columns and fill the columns with 0 #
for g in new_genre_columns:
    top_movies[g] = 0

# pre-clean original genre ids #
top_movies['genre_ids'] = top_movies['genre_ids'].astype(str).apply(lambda a : a.strip(']').strip('['))
top_movies.tail()


# top_movies.columns
# top_movies.shape

,adult,backdrop_path,genre_ids,movie_id,original_language,original_title,overview,popularity,poster_path,release_date,...,36_History,27_Horror,10402_Music,9648_Mystery,10749_Romance,878_Science_Fiction,10770_TV_Movie,53_Thriller,10752_War,37_Western
9995,False,/zhLcgqoIDvxNZPtvQ1PYloGalXs.jpg,"53, 27, 9648",61012,en,Silent Hill: Revelation 3D,Heather Mason and her father have been on the ...,41.568,/xBC7bHZ5UglTw4LMd2lTSgXvQXl.jpg,2012-10-10,...,0,0,0,0,0,0,0,0,0,0
9996,False,/g9ekJY8zNxQ9J3cRYQHzRwiObuq.jpg,"10749, 35",11812,en,The Sweetest Thing,Christina's love life is stuck in neutral. Aft...,15.915,/qLysz4B5YmiDq52Op0zq4eD75LC.jpg,2002-04-12,...,0,0,0,0,0,0,0,0,0,0
9997,False,/eqZ5t82asfprFcNS56eVruNT5G5.jpg,"10751, 35",10481,en,102 Dalmatians,Get ready for a howling good time as an all ne...,18.685,/dSxnIika9yWwTvEbpsmoGdeh65E.jpg,2000-11-21,...,0,0,0,0,0,0,0,0,0,0
9998,False,/gdF4eUHLeEizqTdSGoqfWPIdDX6.jpg,35,450179,fr,Problemos,"When a pandemic strikes the world, Victor and ...",6.521,/12KzS7zZ2Kj4p556lvR7UtJsQoT.jpg,2017-05-10,...,0,0,0,0,0,0,0,0,0,0
9999,False,/1u7TuvR5mSUz2xRQhz01i75IwdT.jpg,"80, 53, 28",201088,en,Blackhat,"Nicholas Hathaway, a furloughed convict, and h...",32.736,/7zu6HPnKPjNA7NuR4DGRdaH0PdX.jpg,2015-01-13,...,0,0,0,0,0,0,0,0,0,0


### Genre column clean up end ------------------------------

were you left off update:

you got the genre column data cleaned up enough to where you can loop through each row, extract the genre data from the original row and place it in the correct new column

you are trying to visualize data BUTTTTTTT the genre column has multiple ids per movie and its stored a list thats been turned into a string and you cant undo that ....so far

... you did that girl :snaps:



In [42]:
most_important_cols = ['genre_ids',
'28',
'12',
'16',
'35',
'80',
'99',
'18',
'10751',
'14',
'36',
'27',
'10402',
'9648',
'10749',
'878',
'10770',
'53',
'10752',
'37']

In [43]:
baby_df = top_movies.head(6)
test = top_movies.head(4).copy()[most_important_cols]
print(test)

  genre_ids  28  12  16  35  80  99  18  10751  14  36  27  10402  9648  \
0    18, 80   0   0   0   0   0   0   0      0   0   0   0      0     0   
1    18, 80   0   0   0   0   0   0   0      0   0   0   0      0     0   
2    35, 14   0   0   0   0   0   0   0      0   0   0   0      0     0   
3    18, 80   0   0   0   0   0   0   0      0   0   0   0      0     0   

   10749  878  10770  53  10752  37  
0      0    0      0   0      0   0  
1      0    0      0   0      0   0  
2      0    0      0   0      0   0  
3      0    0      0   0      0   0  


In [ ]:
# df_size = test.shape[0]

# for i in range(df_size):
#     # test.iloc[[row]] # you have an individual row whoop whoop
#     # take the string of genres and make it a list
#     row = test.iloc[[i]].copy() # do all your work on a copy of the row to avoid a warning
#     genre_str = row['genre_ids'].to_string().split('    ')
#     genre_str.pop(0)
#     genre_list = genre_str[0].split(', ')
    
#     # go through the list of genres 
#     for genre in genre_list:
#         # and for each genre in the list update the matching column
#         row[genre] = str(genre) 
#         test.iloc[[i]] = row #set the orginal row equal to the copy


df_size = test.shape[0]

for i in range(df_size):
    # test.iloc[[row]] # you have an individual row whoop whoop
    # take the string of genres and make it a list
    row = test.iloc[[i]].copy() # do all your work on a copy of the row to avoid a warning
    genre_str = row['genre_ids'].to_string().split('    ')
    genre_str.pop(0)
    genre_list = genre_str[0].split(', ')
    
    # go through the list of genres 
    for genre in genre_list:
        # and for each genre in the list update the matching column
        row[genre] = str(genre) 
        test.iloc[[i]] = pd.Series(row) #set the orginal row equal to the copy


In [ ]:
[pd.Series([x]) for x in test]

In [46]:
# things i can abstract
def wash_genre_ids(df):
    genre_str = df['genre_ids'].to_string().split('    ')
    genre_str.pop(0)
    genre_list = genre_str[0].split(', ')
    return genre_list

wash_genre_ids(test)

['18', '80\n1']

In [ ]:
# can i do this with list comprehension?.....


In [ ]:
# test


In [ ]:
# test['35'].unique()
# pd.Series({c: df[c].unique() for c in df}) new cool thing !!! check for unique values in the whole column instead of just 1 column
pd.Series({c: test[c].unique() for c in test})

In [ ]:
baby_df.loc[baby_df['genre_ids'] == '35, 14']

In [ ]:
def dosomething(genre_items):
    df = baby_df
    if type(genre_items) == str:
        df.loc[df['genre_ids'] == genre_items]['genre_ids']

In [ ]:
# update_genre_column(top_movies.head(2),'36')
[dosomething(genre) for genre in baby_df['genre_ids']]

where you left off 

you're basically done but for some reason your changes aren't being saved in the final data frame

# NEW plan:

## use regex! thanks to Hannah 💃

opt 1: regex for each genre and when its found update that genre's column
opt 2: regex replaceing the current slice work done and still do a loop


In [52]:
len(most_important_cols)
most_important_cols.sort()
most_important_cols

['10402',
 '10749',
 '10751',
 '10752',
 '10770',
 '12',
 '14',
 '16',
 '18',
 '27',
 '28',
 '35',
 '36',
 '37',
 '53',
 '80',
 '878',
 '9648',
 '99',
 'genre_ids']

In [53]:
df_regex_test = top_movies.head(5)
df_regex_test

,adult,backdrop_path,genre_ids,movie_id,original_language,original_title,overview,popularity,poster_path,release_date,...,36,27,10402,9648,10749,878,10770,53,10752,37
0,False,/tmU7GeKVybMWFButWEGl2M4GeiP.jpg,"18, 80",238,en,The Godfather,"Spanning the years 1945 to 1955, a chronicle o...",119.411,/3bhkrj58Vtu7enYsRolD1fZdja1.jpg,1972-03-14,...,0,0,0,0,0,0,0,0,0,0
1,False,/wPU78OPN4BYEgWYdXyg0phMee64.jpg,"18, 80",278,en,The Shawshank Redemption,Framed in the 1940s for the double murder of h...,108.339,/hBcY0fE9pfXzvVaY4GKarweriG2.jpg,1994-09-23,...,0,0,0,0,0,0,0,0,0,0
2,False,/ejniJnlOdtSgtbh8D7u2RxT6Uli.jpg,"35, 14",772071,es,Cuando Sea Joven,70-year-old Malena gets a second chance at lif...,59.051,/6gIJuFHh5Lj4dNaPG3TzIMl7L68.jpg,2022-09-14,...,0,0,0,0,0,0,0,0,0,0
3,False,/kGzFbGhp99zva6oZODW5atUtnqi.jpg,"18, 80",240,en,The Godfather Part II,In the continuing saga of the Corleone crime f...,74.914,/hek3koDUyRQk7FIhPXsa6mT2Zc3.jpg,1974-12-20,...,0,0,0,0,0,0,0,0,0,0
4,False,/zb6fM1CX41D9rF9hdgclu0peUmy.jpg,"18, 36, 10752",424,en,Schindler's List,The true story of how businessman Oskar Schind...,63.594,/sF1U4EUQS8YHUYjNl3pMGNIQyr0.jpg,1993-12-15,...,0,0,0,0,0,0,0,0,0,0


In [55]:
# Regex Approach 💃
# Da Plan...
# use np.select(condition , values) to update the new genre colunms 
# conditions would be using regex and checking if "???" genre is 
    # in the string value found in the orifinal genre column
# the values for each condition would be the genre id or 1 if that doesn't work 

conditions = [
    (df_regex_test['genre_ids'].str.contains(r'18')) # come back later, much later and try to use .loc[row_indexer,col_indexer] = value
]

values = ['18']

df_regex_test['new_genre'] = np.select(conditions,values)


/var/folders/vz/tmrq6vw91zv93d7hk7lpvt1m0000gn/T/ipykernel_870/1624039786.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_regex_test['new_genre'] = np.select(conditions,values)


In [57]:
df_regex_test[['genre_ids','new_genre']]

,genre_ids,new_genre
0,"18, 80",18
1,"18, 80",18
2,"35, 14",0
3,"18, 80",18
4,"18, 36, 10752",18
